In [1]:
from multimodalrec.multimodalrec import MultimodalRec
from multimodalrec.multimodalrec import data_pipeline
from multimodalrec.model import model

import numpy as np
import pandas as pd
import tensorflow as tf

recmodel = MultimodalRec()
recmodel.organize_multimodal_data(load=True)
ratings_df_training = recmodel.user_item_network_training.CF_data
ratings_df_training["Rating"] = ratings_df_training["Rating"]/5. 

Using TensorFlow backend.


Training User-Movie Latent Factors are extracting...
The sparsity level of training dataset is 95.1%


  3%|▎         | 92/2930 [00:00<00:03, 917.30it/s]

Done.
Visual Representations are extracting...


100%|██████████| 2930/2930 [00:03<00:00, 921.81it/s]

Done.


In [2]:
input_1_frames = 30
input_1_dimension = 2048
hidden_size = 5
input_2_dimension = 50
batch_size = 64
is_training = True
dropout = 0.5

In [3]:
tf.reset_default_graph()
with tf.name_scope('main_parameters'):
    x_lstm = tf.placeholder(tf.float32, [None, input_1_frames, input_1_dimension], name='Input_LSTM')
    x_lstm_seq_length = tf.placeholder(tf.int32, [None], name='Input_LSTM_Seq_lenght')
    x_lstm_keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    tf.summary.scalar(x_lstm.op.name, tf.reduce_mean(x_lstm))
    tf.summary.histogram(x_lstm.op.name, tf.reduce_mean(x_lstm))
    
    x_fusion = tf.placeholder(tf.float32, shape=[None, input_2_dimension], name='Input_fusion')
    tf.summary.scalar(x_fusion.op.name, tf.reduce_mean(x_fusion))
    tf.summary.histogram(x_fusion.op.name, tf.reduce_mean(x_fusion))
    
    y = tf.placeholder(tf.float32, shape=[None, 1], name='Output')
    tf.summary.scalar(y.op.name, tf.reduce_mean(y))

In [4]:
x_lstm

<tf.Tensor 'main_parameters/Input_LSTM:0' shape=(?, 30, 2048) dtype=float32>

In [5]:
with tf.variable_scope('frame_level_lstm') as scope:
    #seq_flat = tf.reshape(x_lstm, [-1, input_2_dimension])
    # [batch_size * max_length, hidden_size]
    #seq_embed = tf.contrib.layers.fully_connected(seq_flat, hidden_size)
    seq_embed = tf.reshape(x_lstm, [-1, input_1_frames, hidden_size])
    
    cell = tf.nn.rnn_cell.LSTMCell(hidden_size, state_is_tuple=True)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=x_lstm_keep_prob)
    init_state = cell.zero_state(tf.shape(x_lstm)[0], dtype=tf.float32)

    outputs, states = tf.nn.dynamic_rnn(cell, x_lstm, sequence_length=x_lstm_seq_length,
                                        initial_state=init_state, dtype=tf.float32)
    
    
    batch_range = tf.range(tf.shape(outputs)[0])
    last_index = x_lstm_seq_length - 1
    indices = tf.stack([batch_range, last_index], axis=1)
    last = tf.gather_nd(outputs, indices)
    
    
with tf.variable_scope('concat_level') as scope:
    #user_dense = tf.layers.dense(inputs=x_fusion, units=64, activation=tf.nn.relu)
    concat_tensor = tf.concat(axis=1,values=[last, tf.nn.l2_normalize(x_fusion)])
    #concat_tensor = tf.concat(axis=1,values=[dense2_frames, x_fusion])
    tf.summary.scalar('concat_tensor/bimodal', tf.reduce_mean(concat_tensor))
    tf.summary.histogram('concat_tensor/bimodal', tf.reduce_mean(concat_tensor))
    
with tf.variable_scope('regression_level') as scope:
    layer_1 = tf.layers.dense(inputs=concat_tensor, units=32, activation=tf.nn.relu)
    tf.summary.scalar('layer_1', tf.reduce_mean(layer_1))
    tf.summary.histogram('layer_1', tf.reduce_mean(layer_1))
    
    W_1 = tf.Variable(tf.random_uniform([32,1]))
    b_1 = tf.Variable(tf.zeros([1]))
    pred = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,W_1), b_1))
    #layer_1 = 
      
    #pred = tf.layers.dense(inputs=layer_1, units=1, activation=tf.nn.sigmoid)
    tf.summary.scalar('pred', tf.reduce_mean(pred))
    tf.summary.histogram('pred', tf.reduce_mean(pred))

In [6]:
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)

INFO:tensorflow:Summary name frame_level_lstm/rnn/lstm_cell/kernel:0 is illegal; using frame_level_lstm/rnn/lstm_cell/kernel_0 instead.
INFO:tensorflow:Summary name frame_level_lstm/rnn/lstm_cell/bias:0 is illegal; using frame_level_lstm/rnn/lstm_cell/bias_0 instead.
INFO:tensorflow:Summary name regression_level/dense/kernel:0 is illegal; using regression_level/dense/kernel_0 instead.
INFO:tensorflow:Summary name regression_level/dense/bias:0 is illegal; using regression_level/dense/bias_0 instead.
INFO:tensorflow:Summary name regression_level/Variable:0 is illegal; using regression_level/Variable_0 instead.
INFO:tensorflow:Summary name regression_level/Variable_1:0 is illegal; using regression_level/Variable_1_0 instead.


In [7]:
with tf.variable_scope('loss') as scope:
    cost = tf.sqrt(tf.reduce_mean(tf.square(y - pred)))
    optimizer = tf.train.RMSPropOptimizer(0.1).minimize(cost)#tf.train.AdamOptimizer().minimize(cost)

tf.summary.scalar(cost.op.name, cost)

<tf.Tensor 'loss/Sqrt_1:0' shape=() dtype=string>

In [8]:
sess = tf.Session()

In [9]:
save_dir = "./model/cb_lstm/"
merged = tf.summary.merge_all()

In [10]:
total_parameters = 0
for variable in tf.trainable_variables():
    #print(variable)
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

42905


In [11]:
batch_size_=64
epoch_num = 10

In [ ]:
sess.run(tf.global_variables_initializer())
train_writer = tf.summary.FileWriter(save_dir, sess.graph)

for i in range(epoch_num):
    print('-'*30)
    print('Epoch {}'.format(i))
    print('-'*30)
    
    # for batch in range(int(ratings_df_training.shape[0]/batch_size) + 1):
    for batch in range(30):
        X_train_lstm, X_train_fusion, y_train, indices = data_pipeline(ratings_df_training, recmodel.user_latent_traninig, recmodel.visual_features, 
                                                                       batch_size_, batch, epoch_num=i, random=True)  
        #print(len(y_train[batch * batch_size_: (batch + 1) * batch_size_]))
        batch_lstm_xs = np.array(X_train_lstm, dtype=float)
        batch_train_xs = np.array(X_train_fusion, dtype=float)
        batch_ys = np.array(y_train, dtype=float).reshape(batch_size_,1)
        batch_seq_lenth_xs = np.array([2048]*batch_size)

        #print("{} {} {} {}".format(batch_lstm_xs.shape, batch_train_xs.shape,batch_ys.shape,batch_seq_lenth_xs.shape))


        _, batch_loss = sess.run(
            [optimizer, cost],
            feed_dict={x_lstm: batch_lstm_xs,
                       x_lstm_seq_length:batch_seq_lenth_xs,
                       x_lstm_keep_prob:0.5,
                       x_fusion: batch_train_xs,
                       y: batch_ys})
        msg = "Batch {} - Training Batch Loss: {:.4f}"
        print(msg.format((batch+1), batch_loss))
        
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, optimizer],
                              feed_dict={x_lstm: batch_lstm_xs,
                                           x_lstm_seq_length:batch_seq_lenth_xs,
                                           x_lstm_keep_prob:0.5,
                                           x_fusion: batch_train_xs,
                                           y: batch_ys},
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_summary(summary, batch+(i*30))
        
        
        
        #summary, _ = sess.run([merged, train_step],
        #                      feed_dict=feed_dict(True),
        #                      options=run_options,
        #                      run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % int(batch+(i*30)+1))
        # train_writer.add_summary(summary, i)
        
            
    X_train_lstm, X_train_fusion, y_train, indices = data_pipeline(ratings_df_training, recmodel.user_latent_traninig, recmodel.visual_features, 1000, i, validation=True)  
    #print(len(y_train[batch * batch_size_: (batch + 1) * batch_size_]))
    batch_lstm_xs = np.array(X_train_lstm, dtype=float)
    batch_train_xs = np.array(X_train_fusion, dtype=float)
    batch_ys = np.array(y_train, dtype=float).reshape(3000,1)
    batch_seq_lenth_xs = np.array([2048]*batch_size)
    predictions = sess.run(
        [pred],
        feed_dict={x_lstm: batch_lstm_xs,
                   x_lstm_seq_length:batch_seq_lenth_xs,
                   x_lstm_keep_prob:0.5,
                   x_fusion: batch_train_xs,
                   y: batch_ys})

    RMSE = np.sqrt(np.mean((predictions[0]-y_train)**2))

    msg = "Epoch {} - RMSE: {}"
    print(msg.format((i+1), RMSE))
    
train_writer.close()

------------------------------
Epoch 0
------------------------------
